# Data Engineering and analysis


In [10]:
import pandas as pd


# Load CSV files into pandas dataframes
companies_df = pd.read_csv('../data/companies.csv', dtype=str)
int_cols = ["company_size", "revenue"]
for col in int_cols:
    companies_df[col] = (
        pd.to_numeric(companies_df[col], errors="coerce")  # non-numeric -> NaN
        .astype("Int64")                              # NaN -> <NA>, dtype nullable Int64
    )



revenue_df = pd.read_csv('../data/revenue.csv', dtype=str)
revenue_df["product_sum"] = revenue_df["product_sum"].astype("Int64")
revenue_df["product_duration"] = revenue_df["product_duration"].astype("Int64")
revenue_df['sale_date'] = pd.to_datetime(
    revenue_df['sale_date'],
    errors='coerce'
)

# Display first few rows of each dataframe
print("Companies DataFrame:")
print(companies_df.head())

print("\nRevenue DataFrame:")
print(revenue_df.head())

# Display basic information about dataframes
print("\nCompanies DataFrame Info:")
print(companies_df.info())

print("\nRevenue DataFrame Info:")
print(revenue_df.info())


Companies DataFrame:
  Unnamed: 0         company_id            sector              segment  \
0          0  comp_df0a0c0bd619  sec_07f5ebaa60c6  Public_Small_Medium   
1          1  comp_1e8c0f0ab24c  sec_9d370c38febe       Small_business   
2          2  comp_7b5d176f09b3  sec_5fb21c55bb11       Business_Scale   
3          3  comp_d07c716a7b02  sec_1bc3700e30b8  Public_Small_Medium   
4          4  comp_ca8821847ff0  sec_1bc3700e30b8         Public_Large   

           industry  company_size  revenue  
0  ind_d2091f85a3a3            50   105402  
1  ind_6be1923b9422            28   107905  
2  ind_5fb21c55bb11          5000   248435  
3  ind_11d103ed33f7           100   111632  
4  ind_5ffda163ccd7            50   106819  

Revenue DataFrame:
  Unnamed: 0            row_id         company_id           sale_id  \
0          0  row_8ae3f0a90135  comp_94b3dbfa3cdb  sal_117719067179   
1          1  row_e056ebe0bd07  comp_94b3dbfa3cdb  sal_8f5df72ac2f7   
2          2  row_04471fe231b0 

In [31]:
# LTV клиента = сумма product_sum по company_id
revenue_clean = revenue_df.dropna(subset=['sale_date'])

ltv_df = (
    revenue_clean
    .groupby('company_id')
    .agg(
        first_purchase=('sale_date', 'min'),
        last_purchase=('sale_date', 'max'),
        ltv=('product_sum', 'sum')         # это твой "обычный" LTV
    )
    .reset_index()
)
snapshot_date = revenue_clean['sale_date'].max()
# 4. Возраст клиента в днях
ltv_df['customer_age_days'] = (snapshot_date - ltv_df['first_purchase']).dt.days

# 5. Возраст в годах (с плавающей точкой)
ltv_df['customer_age_years'] = ltv_df['customer_age_days'] / 365.25

# 6. Годовой LTV (annualized LTV)
# Чтобы не взрывался для клиентов с возрастом 0 дней, заменим 0 на NaN
ltv_df.loc[ltv_df['customer_age_years'] <= 0, 'customer_age_years'] = pd.NA

ltv_df['ltv_per_year'] = ltv_df['ltv'] / ltv_df['customer_age_years']

# 7. Приводим типы (где возможно) к nullable Int64
ltv_df['customer_age_days']  = ltv_df['customer_age_days'].astype('Int64')
ltv_df['ltv']                = ltv_df['ltv'].astype('Int64')

ltv_df.head()


,company_id,first_purchase,last_purchase,ltv,customer_age_days,customer_age_years,ltv_per_year
0,comp_0004b685d8d6,2019-05-03,2019-05-03,17000,1624,4.446270,3823.429803
1,comp_001eac7794df,2020-09-23,2023-09-03,270000,1115,3.052704,88446.188341
2,comp_002f5166d79b,2020-11-28,2020-11-28,31000,1049,2.872005,10793.851287
3,comp_00323bea932a,2020-08-17,2020-08-17,16000,1152,3.154004,5072.916667
4,comp_003be29c3721,2022-09-24,2022-09-24,30000,384,1.051335,28535.15625


In [32]:
ltv_df.to_csv('../data/ltv.csv', index=False)


In [19]:
# Берём только строки с валидной датой
rfm_source = revenue_df.dropna(subset=['sale_date'])

# Дата "среза" (анализируем на последний день, который есть в данных)
snapshot_date = rfm_source['sale_date'].max()

# 5. Считаем RFM по company_id
rfm_df = (
    rfm_source
    .groupby('company_id')
    .agg(
        first_purchase=('sale_date', 'min'),
        last_purchase=('sale_date', 'max'),        # дата последней покупки
        frequency=('sale_id', 'nunique'),          # число уникальных заказов
        monetary=('product_sum', 'sum')            # суммарная выручка
    )
    .reset_index()
)

# Recency в днях
rfm_df['recency'] = (snapshot_date - rfm_df['last_purchase']).dt.days

# "Возраст" клиента: сколько дней с первой покупки до snapshot_date
rfm_df['customer_age_days'] = (snapshot_date - rfm_df['first_purchase']).dt.days

# Приводим к Int
rfm_df['recency'] = rfm_df['recency'].astype('Int64')
rfm_df['frequency'] = rfm_df['frequency'].astype('Int64')
rfm_df['monetary'] = rfm_df['monetary'].astype('Int64')

# Оставляем только нужные поля, при необходимости
# rfm_df = rfm_df[['company_id', 'recency', 'frequency', 'monetary']]


In [21]:
# Сохраняем результат
rfm_df.to_csv('../data/rfm.csv', index=False)


In [18]:

# Преобразуем sale_date в datetime, если ещё не
revenue_df['sale_date'] = pd.to_datetime(revenue_df['sale_date'], errors='coerce')

# Базовые временные признаки
revenue_df['sale_year']       = revenue_df['sale_date'].dt.year
revenue_df['sale_month']      = revenue_df['sale_date'].dt.month
revenue_df['sale_day']        = revenue_df['sale_date'].dt.day
revenue_df['sale_quarter']    = revenue_df['sale_date'].dt.quarter
revenue_df['sale_dayofweek']  = revenue_df['sale_date'].dt.dayofweek  # 0=Mon, 6=Sun
revenue_df['is_weekend']      = revenue_df['sale_dayofweek'] >= 5

# Удобный агрегат «год-месяц»
revenue_df['sale_year_month'] = revenue_df['sale_date'].dt.to_period('M').astype(str)

time_cols = ['sale_year', 'sale_month', 'sale_day',
             'sale_quarter', 'sale_dayofweek']

for col in time_cols:
    revenue_df[col] = (
        revenue_df[col]
        .astype('Int64')
    )


revenue_df.head()



In [ ]:
revenue_df.to_csv('../data/revenue-by-sale-date.csv', index=False)

In [24]:
# Кол-во уникальных продуктов
product_count = (
    revenue_df
    .groupby('company_id')['product_name']
    .nunique()
    .reset_index()
    .rename(columns={'product_name': 'n_products'})
)

# Кол-во покупок по типам sale_type
sale_type_counts = (
    revenue_df
    .pivot_table(index='company_id',
                 columns='sale_type',
                 values='row_id',
                 aggfunc='count',
                 fill_value=0)
    .reset_index()
)

# Сборка фичей для кластеризации
features_df = (
    rfm_df
    .merge(product_count, on='company_id', how='left')
    .merge(sale_type_counts, on='company_id', how='left')
)

int_cols = ["Возврат", "Новая", "Продление", "Расширение"]
for col in int_cols:
    features_df[col] = (
        pd.to_numeric(features_df[col], errors="coerce")  # non-numeric -> NaN
        .astype("Int64")                              # NaN -> <NA>, dtype nullable Int64
    )
features_df.head()


,company_id,first_purchase,last_purchase,frequency,monetary,recency,customer_age_days,n_products,Возврат,Новая,Продление,Расширение
0,comp_0004b685d8d6,2019-05-03,2019-05-03,1,17000,1624,1624,1,0,1,0,0
1,comp_001eac7794df,2020-09-23,2023-09-03,27,270000,40,1115,1,0,1,11,15
2,comp_002f5166d79b,2020-11-28,2020-11-28,1,31000,1049,1049,2,0,3,0,0
3,comp_00323bea932a,2020-08-17,2020-08-17,1,16000,1152,1152,1,0,1,0,0
4,comp_003be29c3721,2022-09-24,2022-09-24,1,30000,384,384,3,0,3,0,0


In [25]:
features_df.to_csv('../data/rfm-features.csv', index=False)

In [28]:
# Матрица company_id x product_name (1 если клиент покупал этот продукт)
basket = (
    revenue_df
    .assign(value=1)
    .pivot_table(index='company_id',
                 columns='product_name',
                 values='value',
                 aggfunc='max',
                 fill_value=0)
)
basket.head()

product_name,Enterprise Total-1000,Enterprise Total-150,Enterprise Total-2000,Enterprise Total-300,Enterprise Total-3000,Enterprise Total-500,Enterprise Total-5000,Enterprise-100,Enterprise-1000,Enterprise-150,...,Дополнительные услуги,Доски,МТС Линк. Команда,МТС Линк. Компания,МТС Линк. Корпорация,Общие Тарифы Webinar Enterprise,Расширения,Тарифы TMS,Тарифы Webinar СМБ,Чаты
company_id,,,,,,,,,,,,,,,,,,,,,
comp_0004b685d8d6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
comp_001eac7794df,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
comp_002f5166d79b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
comp_00323bea932a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
comp_003be29c3721,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1


In [30]:
basket.to_csv('../data/basket.csv', index=True)